Modified: Jul 17, 2019
# Curve Representation using Levelset function
- Signed Distance Function


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import holoviews as hv
from holoviews import opts, dim, streams

import panel as pn
hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
import streamz
import streamz.dataframe

To deinf a curve on a plane (ie. planary curve) we need
- parameter, eg. p $\in [0,1]$
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

## Signed Distance Function 
### For a circle

- $\phi(x,y)$ is the distance between point $p = (x,y)$ and the curve. Here the curve we want to represent is a unit circle
- $\phi(x,y)$ is Dist(origin, $p$) - Dist(origin, a point on the unit circle touched by a stright ray passing through the origin and $p$). The second term is always 1 for a unit circle. So we get the $\phi$ as following:

$ \phi(x,y) = \sqrt{(x^2+y^2)} - 1 $

Now let's visualize it using holoviews.

In [ ]:
Y,X = np.mgrid[-1:1:0.5, -2:2:0.5]

In [ ]:
Y.shape, X.shape

In [ ]:
Y,X

In [ ]:
def phi(x,y):
    return np.sqrt(x**2 + y**2) - 1


In [ ]:
Z = phi(X,Y)


In [ ]:
hv.Scatter3D( (X.flat, Y.flat, Z.flat)).opts(azimuth=40, elevation=20, color='z', s=50, cmap='fire')

In [ ]:
X.flat

In [ ]:
Z

In [ ]:
y,x = np.mgrid[-5:5, -5:5] * 0.1
heights = np.sin(x**2+y**2)
hv.Scatter3D((x.flat, y.flat, heights.flat)).opts(
    opts.Scatter3D(azimuth=40, elevation=20, color='z', s=50, cmap='fire'))

In [ ]:
(hv.Scatter3D(np.random.randn(100,4), vdims='Size') * hv.Scatter3D(np.random.randn(100,4)+2, vdims='Size')).opts(
    opts.Scatter3D(s=(5+dim('Size'))*10, marker='^'))

In [ ]:
hv.__version__

In [ ]:
class CurveSimulator(param.Parameterized):

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.)

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,width=W, height=H, 
                             xlim=(-1,1), ylim=(-1,1),
                             color=dim('p')*256-50,
                             tools=['hover']
                            )
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        point = pd.DataFrame([(self.p, *self.cfunc(self.p))], columns=['p','x','y'])
        self.data_src.emit(point)
        time.sleep(self.t_interval)
        
    def reset_handler(self):
        self.set_param(p=0.0)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, cfunc, n_steps=100, **kwargs):
        """
        Args:
        - cfunc (function): given an input of a float p in [0,1], returns (x,y), a 
        tuple of x and y coords
        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.cfunc = cfunc 
        self.n_steps = n_steps
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=min(self.n_steps, 100), index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    

    def set_dmap_curve(self):
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_x(self):
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        


### Define curve function

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Try different functions
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)**10
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Alternating along a straight line 
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)**2

In [ ]:
# Doesn't have to choose a periodic function
xfunc = lambda p: np.log(p)
yfunc = lambda p: p

In [ ]:
# Something happens at p=0.5
xfunc = lambda p: np.sin(2*np.pi*p**2)*p**3
yfunc = lambda p: np.sin(np.pi*p**0.5)

### Create the simulator for the curve

In [ ]:
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

### Show the simulator

In [ ]:
pn.Row(
    pn.Param(c.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button(name=c.param['reset'].label),
        't_interval': pn.widgets.FloatSlider
    }),
    pn.panel(c.viewable())
)

In [ ]:
c.dfstream.data